# Code for running inference

To convert this to a script, should have a name for the whole training task which goes in dataset.json and/or datalist.json because this will help for the task.json needed for inference.

Look into the minimal amount of metaata monai requires in the form of a task.json or other struct to initialize the builder and then run the ensemble

In [7]:
import os
from pathlib import Path
import numpy as np
import json

import mri_data
import monai_training

from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

from reload_recursive import reload_recursive

In [11]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data import utils
from mri_data.file_manager import scan_3Tpioneer_bids, Scan, DataSet, filter_first_ses
from monai_training import preprocess

In [3]:
msmri_home = Path("/home/srs-9/Projects/ms_mri")
training_work_dirs = Path("/mnt/h/training_work_dirs")

In [4]:
dataset_file = training_work_dirs / "choroid_pineal_pituitary1" / "dataset.json"
dataset, info = preprocess.load_dataset(dataset_file)
img = utils.load_nifti(dataset[0].image_path)

In [12]:
dataroot = "/mnt/h/3Tpioneer_bids"
work_dir  = msmri_home / "training_work_dirs" / "pineal1"
train_dataset_file = work_dir / "training-dataset.json"
dataset_train, _ = preprocess.load_dataset(train_dataset_file)

dataset_proc = preprocess.DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses])
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ['subid', 'sesid'])

In [13]:
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(["flair", "t1"])

2024-10-17 02:25:42.898 | INFO     | monai_training.preprocess:prepare_images:88 - Prepare Images
2024-10-17 02:25:52.653 | INFO     | mri_data.utils:merge_images:27 - fslmerge -a /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.t1.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/t1.nii.gz


Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated


2024-10-17 02:26:03.717 | ERROR    | monai_training.preprocess:prepare_images:121 - Something went wrong merging images for <bound method Scan.info of Scan(subid='2120', sesid='20170920', dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920'), image=None, label=None, cond=None, id=42762350400)>
Traceback (most recent call last):

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
    │   └ <bound method Application.launch_instance of <class 'ipykernel.kernelapp.IPKernelApp'>>
    └ <module 'ipykernel.kernelapp' from '/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ipykernel/kernelapp.py'>
  File "/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.s

Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated


2024-10-17 02:26:18.430 | ERROR    | monai_training.preprocess:prepare_images:121 - Something went wrong merging images for <bound method Scan.info of Scan(subid='1196', sesid='20161004', dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004'), image=None, label=None, cond=None, id=24112560784)>
Traceback (most recent call last):

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
    │   └ <bound method Application.launch_instance of <class 'ipykernel.kernelapp.IPKernelApp'>>
    └ <module 'ipykernel.kernelapp' from '/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ipykernel/kernelapp.py'>
  File "/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.s

In [14]:
len(dataset)
with open("/home/srs-9/Projects/ms_mri/datalists/choroid_pineal_pituitary.txt", 'w') as f:
    for scan in dataset:
        f.write(scan.root + "\n")

28

In [21]:
dataset_t1_flair = scan_3Tpioneer_bids(dataroot, image="flair.t1.nii.gz")
dataset_t1_flair = [scan for scan in dataset_t1_flair if scan.image is not None]
dataset_tmp = DataSet.from_scans(dataset_t1_flair)

In [23]:
dataset_tmp = DataSet.from_scans(dataset_t1_flair)
images = []
# for scan in dataset_proc.dataset:
for scan in dataset_tmp:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

datalist = {
    "testing": images
}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f)

task = {
  "name": "infer_pineal",
  "task": "segmentation",
  "modality": "MRI",
  "datalist": str(work_dir / "datalist.json"),
  "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, "inference-task.json")
with open(task_file, "w") as f:
    json.dump(task, f)

In [8]:
input_cfg = work_dir  / "inference-task.json"  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [13]:
save_dir = Path("/mnt/h/3Tpioneer_bids_predictions")

## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage", 
    "output_dir": save_dir, 
    "data_root_dir": dataroot, 
    "output_postfix": "pineal_pred", 
    "separate_folder": False}
                               
pred = ensemble(pred_param = {"image_save_func": save_params})

KeyError: 'query key: dataroot'

In [1]:
cmd_parts = ['fslmerge', '-a', '/media/hemondlab/Data/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.t1.nii.gz', '/media/hemondlab/Data/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.nii.gz', '/media/hemondlab/Data/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz']
cmd = " ".join(cmd_parts)
print(cmd)

fslmerge -a /media/hemondlab/Data/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.t1.nii.gz /media/hemondlab/Data/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.nii.gz /media/hemondlab/Data/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz


In [2]:
path1 = "/media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004" 
path2 = "/media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004"
path1 == path2

False

In [7]:
from pathlib import Path

projects_root = Path("/home/hemondlab/Dev")
drive_root = Path("/media/hemond_lab/Data")

msmri_home = projects_root / "ms_mri"
training_work_dirs = drive_root / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = msmri_home / "training_work_dirs" / "pineal1"
train_dataset_file = work_dir / "training-dataset-desktop1.json"

In [8]:
input_cfg = (
    work_dir / "inference-task.json"
)
input_cfg.exists()

True

In [10]:
import os
path = '/home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/swinunetr_0/model_fold0/progress.yaml'
os.path.exists(path)

False

In [23]:
from typing import Callable

check = scan_3Tpioneer_bids
isinstance(check, Callable)

True

In [20]:
thoo = 'thoo'
isinstance(thoo, str)

True